# Train trip time calculator
Originally writtend by Alon Levy, refined by Paddy Mullen

In [1]:
def integratel(f,a,b,n):
    """  integration function """
    a1 = float(a)
    b1 = float(b)
    y = 0
    for i in range(0,n):
        y = y + f(a1 + (b1-a1)*i/n)
    return y*(b1-a1)/n

def acctfn(k,a,b,c,m):
    return lambda x : max(x/(k - a*x - b*x**2 - c*x**3), 1/m)

def dectfn(k,a,b,c,m):
    return lambda x : max(x/(k + a*x + b*x**2 + c*x**3), 1/m)

def accdfn(k,a,b,c,m):
    return lambda x : max((x**2)/(k - a*x - b*x**2 - c*x**3), x/m)

def decdfn(k,a,b,c,m):
    return lambda x : max((x**2)/(k + a*x + b*x**2 + c*x**3), x/m)

def acctime(k,a,b,c,m,x1,x2,n):
    if x1 <= x2:
        return integratel(acctfn(k,a,b,c,m),x1,x2,n)
    else:
        return integratel(acctfn(k,a,b,c,m),x2,x1,n)

def dectime(k,a,b,c,m,x1,x2,n):
    """ deceleration time """
    if x1 <= x2:
        return integratel(dectfn(k,a,b,c,m),x1,x2,n)
    else:
        return integratel(dectfn(k,a,b,c,m),x2,x1,n)
    
def accdist(k,a,b,c,m,x1,x2,n):
    """accelleration dist """
    if x1 <= x2:
        return integratel(accdfn(k,a,b,c,m),x1,x2,n)
    else:
        return integratel(accdfn(k,a,b,c,m),x2,x1,n)
    
def decdist(k,a,b,c,m,x1,x2,n):
    """ decelleration distance """
    if x1 <= x2:
        return integratel(decdfn(k,a,b,c,m),x1,x2,n)
    else:
        return integratel(decdfn(k,a,b,c,m),x2,x1,n)
    
def accpen(k,a,b,c,m,x1,x2,n):
    """ acceleration penalty """
    return acctime(k,a,b,c,m,x1,x2,n) - accdist(k,a,b,c,m,x1,x2,n)/max(x1, x2)

def decpen(k,a,b,c,m,x1,x2,n):
    """ deceleration penalty """
    return dectime(k,a,b,c,m,x1,x2,n) - decdist(k,a,b,c,m,x1,x2,n)/max(x1, x2)

def slowpen(k,a,b,c,m,x1,x2,n):
    """ slow penalty """
    return accpen(k,a,b,c,m,x1,x2,n) + decpen(k,a,b,c,m,x1,x2,n)

#orig
def speedzone(k,a,b,c,m,u,v,n):
    if len(u) - len(v) != 1:
        return "error, v must have length 1 less than u."
    if min(v) <= 0:
        return "error, all speed zones must be positive."
    r = []
    r.append(0.5*slowpen(k,a,b,c,m,0,v[0],n))
    for i in range(len(v)-1):
        r.append(1000*(u[i+1]-u[i])/v[i])
        r.append(0.5*slowpen(k,a,b,c,m,v[i],v[i+1],n))
    r.append(1000*(u[len(v)]-u[len(v)-1])/v[len(v)-1])
    r.append(0.5*slowpen(k,a,b,c,m,0,v[len(v)-1],n))
    return r

def speedzonek(k,a,b,c,m,u,v,n):
    w = [i/3.6 for i in v]
    return speedzone(k,a,b,c,m,u,w,n)

In [2]:
#speedzone refactored
def speedzone(k,a,b,c,m,u,v,n):
    #u is km_points
    #v is speedzones
    assert v[-1] == 0 #speedzone last has a speed of 0
    assert len(v) == len(u)

    def local_slowpen(vel_0, vel_1):
        return  0.5*slowpen(k,a,b,c,m,vel_0,vel_1,n)
    def main_speed(km_0, km_1, vel):
        return 1000 * (km_1 - km_0)/vel
    
    r = [local_slowpen(0,v[0])]    
    for i in range(len(v)-1):        
        current_vel, current_km = v[i], u[i]
        next_vel, next_km = v[i+1], u[i+1]
        r.append(main_speed(current_km, next_km, current_vel))
        r.append(local_slowpen(current_vel, next_vel))
    return r

In [3]:
def calc_trip_time(train, segments):
    km_points, velocities = zip(*segments)
    updated_velocities = [min(train.max_speed_km, v) for v in velocities]
    
    times = speedzonek(
        train.power_weight, #k
        train.constant_resistance, #a
        train.linear_resistance, #b
        train.quadratic_resistance, #c
        train.initial_accel, #m
        km_points,
        updated_velocities,
        500)
    return times

In [4]:
#Some simple classes to represent different types of trains
class TrainN700:
    power_weight = 26.74 #k
    initial_accel = 0.9 #m
    max_speed_km = 300
    
    constant_resistance = 0.0059 #a
    linear_resistance = 0.000118 # b
    quadratic_resistance = 0.000022 # c
    
class TrainGeneric:
    power_weight = 20
    initial_accel = 0.5
    max_speed_km = 55

    
    constant_resistance = 0.0059 #a
    linear_resistance = 0.000118 # b
    quadratic_resistance = 0.000022 # c

In [5]:
segments = [
    #km-post, speed limit
    (0, 10), #station, limited speed section
    (1, 60), #start of park ave segment,
    (6, 0)]  #125th street station, 0 speed because our segment ends here
speeds = calc_trip_time(TrainN700, segments)
print(speeds)
print(sum(speeds))

[1.546296296296262, 360.0, 6.4429012345677314, 300.0, 9.27777777777757]
677.2669753086416


In [6]:
#same segment with a generic train, max speed of 55
speeds = calc_trip_time(TrainGeneric, segments)
print(speeds)
print(sum(speeds))

[2.7833333333333337, 360.0, 10.24772727272729, 327.2727272727273, 15.308333333333334]
715.6121212121212


In [7]:
#300 km/h to km 5
segments = [
    #km-post, speed limit
    (0, 300), #station, limited speed section
    (5, 20), #start of park ave segment,
    (6, 0)]  #125th street station, 0 speed because our segment ends here
speeds = calc_trip_time(TrainN700, segments)
print(speeds)
print(sum(speeds))

[59.128869980665286, 60.0, 53.14987931473805, 180.0, 3.092592592592524]
355.37134188799587


In [8]:
#300 km/h to km 5
segments = [
    #km-post, speed limit
    (0, 300), #station, limited speed section
    (1, 300), #start of park ave segment,
    (5, 20), #start of park ave segment,
    (6, 0)]  #125th street station, 0 speed because our segment ends here
speeds = calc_trip_time(TrainN700, segments)
print(speeds)
print(sum(speeds))

[59.128869980665286, 12.0, 0.0, 48.0, 53.14987931473805, 180.0, 3.092592592592524]
355.37134188799587


# Calc check
Note that above cell has 4 segments, but since the speed limit is the same for two consecutive segments, the total time should remain constant... which it does

In [9]:
#even with the high speed, the TrainGeneric is speed limited
speeds = calc_trip_time(TrainGeneric, segments)
print(speeds)
print(sum(speeds))

[15.308333333333334, 65.45454545454545, 0.0, 261.8181818181818, 6.1992424242424224, 180.0, 5.566666666666667]
534.3469696969697


In [10]:
#what if we make this 300 km/h right out of the gate
#also does this code work with only two segments
segments = [
    #km-post, speed limit
    (0, 300), #station, limited speed section
    (6, 0)]  #125th street station, 0 speed because our segment ends here
speeds = calc_trip_time(TrainN700, segments)
print(speeds)
print(sum(speeds))

[59.128869980665286, 72.0, 59.128869980665286]
190.25773996133057


In [11]:
! cat Trains-alon.ipynb | pbcopy